# 03 - 数据处理

## 学习目标

- 掌握分组聚合操作（GroupBy）
- 学会连接操作（Join）
- 学会处理缺失值
- 掌握数据类型转换
- 了解窗口函数的使用

In [ ]:
import daft
from daft import col

df = daft.read_parquet("../data/products.parquet")

## 1. 聚合和分组

### 1.1 按类别统计

In [ ]:
# 按类别统计：平均价格、最高价格、产品数量
stats = df.groupby("category").agg(
    col("price").mean().alias("avg_price"),
    col("price").max().alias("max_price"),
    col("price").count().alias("count"),
)
stats.show()

### 1.2 多列分组

In [ ]:
# 按类别 + 品牌统计
stats2 = df.groupby("category", "brand").agg(
    col("rating").mean().alias("avg_rating"),
    col("product_id").count().alias("count"),
)
stats2.sort("count", desc=True).show(10)

### 1.3 多种聚合函数

Daft 支持的常用聚合函数：`count()`, `sum()`, `mean()`, `min()`, `max()`。

In [ ]:
# 综合聚合示例
df.groupby("category").agg(
    col("price").min().alias("min_price"),
    col("price").max().alias("max_price"),
    col("price").mean().alias("avg_price"),
    col("price").sum().alias("total_price"),
    col("product_id").count().alias("product_count"),
).show()

## 2. 连接操作

### 2.1 内连接（Inner Join）

将原始数据与类别统计信息连接，为每行添加类别级别的统计。

In [ ]:
# 内连接：原始数据 + 类别统计
category_stats = df.groupby("category").agg(
    col("price").mean().alias("avg_price"),
    col("price").max().alias("max_price"),
    col("price").count().alias("count"),
)
df_joined = df.join(category_stats, on="category", how="inner")
df_joined.select("name", "category", "price", "avg_price", "max_price").show(5)

### 2.2 左连接（Left Join）

In [ ]:
# 构造两个 DataFrame 演示左连接
df_products = daft.from_pydict({
    "product_id": ["P001", "P002", "P003", "P004"],
    "name": ["手机", "电脑", "耳机", "平板"],
})

df_reviews = daft.from_pydict({
    "product_id": ["P001", "P002", "P005"],
    "review_score": [4.5, 4.8, 3.9],
})

# 左连接：保留所有产品，即使没有评论
df_left = df_products.join(df_reviews, on="product_id", how="left")
df_left.show()

### 2.3 多键连接

In [ ]:
# 多键连接示例
df_a = daft.from_pydict({
    "category": ["电子产品", "电子产品", "服装"],
    "brand": ["华为", "小米", "耐克"],
    "sales": [1000, 800, 500],
})

df_b = daft.from_pydict({
    "category": ["电子产品", "电子产品", "服装"],
    "brand": ["华为", "苹果", "耐克"],
    "market_share": [0.3, 0.25, 0.15],
})

df_multi_key = df_a.join(df_b, on=["category", "brand"], how="inner")
df_multi_key.show()

## 3. 处理缺失值

### 3.1 统计缺失值

In [ ]:
# 查看哪些列有缺失值
# 使用 is_null() 统计每列的 null 数量
df.agg(
    col("brand").is_null().cast(daft.DataType.int64()).sum().alias("brand_nulls"),
    col("rating").is_null().cast(daft.DataType.int64()).sum().alias("rating_nulls"),
    col("description").is_null().cast(daft.DataType.int64()).sum().alias("description_nulls"),
    col("product_id").count().alias("total_rows"),
).show()

### 3.2 填充缺失值 - `fill_null()`

In [ ]:
# 填充缺失值
df_filled = df.with_columns({
    "brand": col("brand").fill_null("未知品牌"),
    "rating": col("rating").fill_null(0.0),
})

# 验证填充结果
df_filled.agg(
    col("brand").is_null().cast(daft.DataType.int64()).sum().alias("brand_nulls"),
    col("rating").is_null().cast(daft.DataType.int64()).sum().alias("rating_nulls"),
).show()

In [ ]:
# 查看填充后的数据（筛选原来 brand 为空的行）
df_filled.where(col("brand") == "未知品牌").select("product_id", "name", "brand", "rating").show(5)

### 3.3 过滤非空行 - `is_not_null()`

In [ ]:
# 只保留有评分的产品
df_with_rating = df.where(col("rating").not_null())
print(f"有评分的产品数: {len(df_with_rating.collect())}")
print(f"总产品数: {len(df.collect())}")

## 4. 数据类型转换

### 4.1 查看当前 Schema

In [ ]:
print("转换前 Schema:")
print(df.schema())

### 4.2 使用 `cast()` 转换类型

In [ ]:
# 将 price 转为 float32，review_count 转为 int32
df_casted = df.with_columns({
    "price": col("price").cast(daft.DataType.float32()),
    "review_count": col("review_count").cast(daft.DataType.int32()),
})

print("转换后 Schema（关注 price 和 review_count）:")
for field in df_casted.schema():
    if field.name in ("price", "review_count"):
        print(f"  {field.name}: {field.dtype}")

## 5. 窗口函数

使用 `Window` 定义窗口规范，配合 `over()` 在分组内进行计算，而不折叠行。

Window 支持：
- `partition_by()` - 分组
- `order_by()` - 窗口内排序
- `rows_between()` / `range_between()` - 自定义窗口帧

In [ ]:
# 5.1 分组内聚合
from daft import Window

window_spec = Window().partition_by("category")

# 计算每个类别内的平均价格，并添加为新列
df_window = df.with_column(
    "category_avg_price",
    col("price").mean().over(window_spec),
)

df_window.select("name", "category", "price", "category_avg_price").show(10)

In [ ]:
# 计算每个产品价格与类别平均价格的差异
df_diff = df_window.with_column(
    "price_vs_avg",
    col("price") - col("category_avg_price"),
)

df_diff.select("name", "category", "price", "category_avg_price", "price_vs_avg").sort("price_vs_avg", desc=True).show(10)

### 5.2 窗口排序与 lag()

使用 `order_by()` 指定窗口内排序，配合 `lag()` 访问前一行的值。

In [ ]:
# 在类别内按价格排序，计算相邻产品的价格差异
window_ordered = Window().partition_by("category").order_by("price")

df_lag = df.select("name", "category", "price").with_column(
    "prev_price",
    col("price").lag(1, default=None).over(window_ordered),
).with_column(
    "price_delta",
    col("price") - col("prev_price"),
)

df_lag.sort(["category", "price"]).show(10)

## 总结

本节学习了 Daft 的数据处理操作：

| 操作 | 方法 | 说明 |
|------|------|------|
| 分组聚合 | `groupby().agg()` | 按列分组并计算统计量 |
| 内连接 | `join(on=, how="inner")` | 保留两表都有的行 |
| 左连接 | `join(on=, how="left")` | 保留左表所有行 |
| 填充空值 | `fill_null()` | 用指定值替换 null |
| 过滤空值 | `is_not_null()` | 过滤非空行 |
| 类型转换 | `cast()` | 转换列的数据类型 |
| 窗口函数 | `expr.over(partition)` | 分组内计算，不折叠行 |

## 练习题

1. 按 `subcategory` 分组，计算每个子类别的平均评分和产品数量
2. 创建一个品牌信息表，与产品表进行左连接
3. 使用窗口函数计算每个品牌内的价格排名

## 下一步

继续学习 [04_advanced_features.ipynb](./04_advanced_features.ipynb) —— 掌握 UDF、查询计划和性能优化。